#### 1. FRED API To Download USA Economic Data - GDP, CPI ETC.

In [1]:
import requests
import pandas as pd

# Define your FRED API key
API_KEY = "dc6a0313beabf8a2845ed521ddb56d7b"

# Define the FRED API endpoint for FOMC meeting dates
FRED_ENDPOINT = 'https://api.stlouisfed.org/fred/series/observations'

# Parameters for the API request
params = {
    'series_id': 'FEDFUNDS',  # This series represents the federal funds rate, which is influenced by FOMC meetings
    'api_key': API_KEY,
    'file_type': 'json'
}

# Make the API request
response = requests.get(FRED_ENDPOINT, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Extract observation dates
    observations = data['observations']
    
    
    # Create a DataFrame
    #df = pd.DataFrame(dates, columns=['Date'])
    df = pd.DataFrame(observations)
    df = df[['date', 'value']]
    df.columns = ['date', 'ffr']
    df['date'] = pd.to_datetime(df['date'])
    # Print the DataFrame
    print(df)
else:
    print(f"Error fetching data: {response.status_code}")

# Optionally, save the data to a CSV file
#df.to_csv('fomc_meeting_dates.csv', index=False)


          date   ffr
0   1954-07-01  0.80
1   1954-08-01  1.22
2   1954-09-01  1.07
3   1954-10-01  0.85
4   1954-11-01  0.83
..         ...   ...
837 2024-04-01  5.33
838 2024-05-01  5.33
839 2024-06-01  5.33
840 2024-07-01  5.33
841 2024-08-01  5.33

[842 rows x 2 columns]


#### Helper Functions

In [3]:
def complete_month(month_abbr):
  """Completes a month name from its abbreviation.

  Args:
    month_abbr: The first three letters of a month name.

  Returns:
    The complete month name if found, otherwise None.
  """

  for month_number, month_name in enumerate(calendar.month_name):
    if month_name.lower().startswith(month_abbr.lower()):
      return month_name
  return None

def month_str_to_num(month_str):
  """Converts a month string to its corresponding number.

  Args:
    month_str: The month name as a string (e.g., "January", "Feb").

  Returns:
    The month number (1-12) if found, otherwise None.
  """

  month_str = month_str.lower()
  for month_num, month_name in enumerate(calendar.month_name):
    if month_name.lower().startswith(month_str):
      return month_num
  return None

#### 2. Web Scrape FED Website For Future FOMC Meetings

In [4]:
from bs4 import BeautifulSoup
import calendar
import datetime
import re 
import requests

year = 2024 # in your code take the today date and extract the year to automate the entire process 
year = 2024 # in your code take the today date and extract the year to automate the entire process 

# URL of the FOMC meeting calendar page
url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the header with the text "2024 FOMC Meetings"
    header = soup.find('h4', string=f"{year} FOMC Meetings")

    parent_div1 = header.find_parent('div')

    main_parent= parent_div1.find_parent('div')

    months = main_parent.find_all('div', re.compile("fomc-meeting__month "))
    days = main_parent.find_all('div', "fomc-meeting__date col-xs-4 col-sm-9 col-md-10 col-lg-1")
    
    dates = {}
    for i in range(len(months)):
        
        month = months[i].text
        
        if '/' in month:
            m1, m2 = month.split("/")
            d1, d2 = days[i].text.split('-')
            
            #m1 = complete_month(m1)
            m2 = complete_month(m2)
            
            #dates[m1] = d1 # just remove this line if not interested in day 1 of the FED 
            dates[m2] = d2
            
            continue
        
        else:
            dates[months[i].text] = days[i].text.split('-')

    #dates = pd.DataFrame(dates, index = [1,2] ,columns = dates.keys())
    
dates  
    

{'January': ['30', '31'],
 'March': ['19', '20*'],
 'May': '1',
 'June': ['11', '12*'],
 'July': ['30', '31'],
 'September': ['17', '18*'],
 'November': ['6', '7'],
 'December': ['17', '18*']}

In [5]:
today = datetime.date.today()
month = calendar.month_name[today.month]
day = today.day
print("today: ", today)
print("Month and Day: ",  (year, month, day ))

today:  2024-09-24
Month and Day:  (2024, 'September', 24)


In [6]:

try:
    days = dates[month]
    print(f'Fed Decision Day on {month}', days, year)
    try:
        #fed_decision_date = datetime.date(year, month_str_to_num(month),int(days[1].replace('*','')))
        fed_decision_date = datetime.date(year, today.month, int(days[1].replace('*','')))
    except IndexError:
        #fed_decision_date = datetime.date(year, month_str_to_num(month),int(days[0].replace('*','')))
        fed_decision_date = datetime.date(year, today.month, int(days[0].replace('*','')))
    
except KeyError:
    print(f'No FED in {month}.')
    next_month = calendar.month_name[(today + datetime.timedelta(days=32)).month]
    days = dates[next_month]
    try:
        fed_decision_date = datetime.date(year, today.month + 1, int(days[1].replace('*','')))
        print(f'Fed Decision on {next_month}', days[1], year)
    except IndexError:
        fed_decision_date = datetime.date(year, today.month + 1, int(days[0].replace('*','')))
        print(f'Fed Decision on {next_month}', days[0], year)

Fed Decision Day on September ['17', '18*'] 2024


In [7]:
import datetime

def count_weekdays(start_date, end_date):
  """Counts the number of weekdays between two dates.

  Args:
    start_date: The start date as a datetime.date object.
    end_date: The end date as a datetime.date object.

  Returns:
    The number of weekdays between the two dates.
  """

  delta = end_date - start_date
  total_days = delta.days + 1  # Include both start and end dates

  # Calculate the number of full weeks
  full_weeks, remaining_days = divmod(total_days, 7)
  weekdays = full_weeks * 5  # 5 weekdays per week

  # Handle the remaining days
  start_weekday = start_date.weekday()
  end_weekday = end_date.weekday()

  if start_weekday < 5:  # Start day is not a weekend
    weekdays += min(end_weekday + 1, 5) - start_weekday
  else:
    weekdays += max(0, end_weekday - 4)

  return weekdays

num_weekdays = count_weekdays(today, fed_decision_date)
print("Trading days until FED decision: ", num_weekdays) 
print('Calendar Days until FED decision: ', (fed_decision_date - today).days)


Trading days until FED decision:  -3
Calendar Days until FED decision:  -6


#### 3. Web Scrape Historical FOMC Meeting Dates 1990 - 2018

In [8]:
import numpy as np 

def get_html_item(url, item):

    # Make a GET request to fetch the raw HTML content
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the header
        headers = soup.find_all(item)

    return headers

# Define a function to extract month, day(s), and year
def extract_date_components(date_str):
    # Extract the month
    month_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December)'
    month_match = re.search(month_pattern, date_str)
    month = month_match.group(0) if month_match else 'None'

    # Extract the day(s)
    day_pattern = r'(\d{1,2}(?:-\d{1,2})?)'
    day_match = re.search(day_pattern, date_str)
    day = day_match.group(0) if day_match else 'None'

    # Extract the year
    year_pattern = r'\b(\d{4})\b'
    year_match = re.search(year_pattern, date_str)
    year = year_match.group(0) if year_match else 'None'

    return month, day, year

In [9]:
start_year = 1990
end_year = 2019

df = pd.DataFrame()

for year in range(start_year, end_year, 1):
    
    # URL of the FOMC meeting calendar page
    url = f"https://www.federalreserve.gov/monetarypolicy/fomchistorical{year}.htm"

    # Make a GET request to fetch the raw HTML content
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the header with the text "2024 FOMC Meetings"
        headers = soup.find_all('h5')
        
        dates_list = []
        
        for header in headers:

            items = header.text
            dates_list.append(items)
            
        dates_list = pd.DataFrame(dates_list)
        df = pd.concat([df, dates_list], axis = 0)
    else: 
        print("Unsuccessful Call: ", response.status_code)


df.columns = ['dates']

df

,dates
0,February 6-7 Meeting - 1990
1,March 27 Meeting - 1990
2,April 11 Conference Call - 1990
3,May 15 Meeting - 1990
4,July 2-3 Meeting - 1990
...,...
3,June 12-13 Meeting - 2018
4,Jul/Aug 31-1 Meeting - 2018
5,September 25-26 Meeting - 2018
6,November 7-8 Meeting - 2018


In [10]:

# Apply the function to the 'dates' column and create new columns
df[['month', 'day', 'year']] = df['dates'].apply(lambda x: pd.Series(extract_date_components(x)))
df = df.reset_index(drop = True)

df['month_day1'] = np.where(df['dates'].str.contains('/'), df['dates'].str.split(' '), 'nan')
df['month_day1'] = df['month_day1'].apply(lambda x: x[0])
df['month_day1'] = np.where(df['dates'].str.contains('/'), df['month_day1'].str.split('/'), 'nan')
df['month_day1'] = df['month_day1'].apply(lambda x: x[0])

df['month_day2'] = np.where(df['dates'].str.contains('/'), df['dates'].str.split(' '), 'nan')
df['month_day2'] = df['month_day2'].apply(lambda x: x[0])
df['month_day2'] = np.where(df['dates'].str.contains('/'), df['month_day2'].str.split('/'), 'nan')
df['month_day2'] = df['month_day2'].apply(lambda x: x[1])

df['day1'] = df['day'].str.split('-').apply(lambda x: x[0] if len(x) > 1 else 'nan')
df['day2'] = df['day'].str.split('-').apply(lambda x: x[1] if len(x)> 1 else x[0])

timestamps = []

for i, row in df.iterrows():
    
    if '/' in row['dates']:
        timestamp = datetime.date(int(row['year']), month_str_to_num(row['month_day2']), int(row['day2']))
        timestamps.append(timestamp)
        
    else: 
        timestamp = datetime.date(int(row['year']), month_str_to_num(row['month']), int(row['day2']))
        timestamps.append(timestamp)

        
df['timestamp'] = timestamps

df

,dates,month,day,year,month_day1,month_day2,day1,day2,timestamp
0,February 6-7 Meeting - 1990,February,6-7,1990,n,a,6,7,1990-02-07
1,March 27 Meeting - 1990,March,27,1990,n,a,nan,27,1990-03-27
2,April 11 Conference Call - 1990,April,11,1990,n,a,nan,11,1990-04-11
3,May 15 Meeting - 1990,May,15,1990,n,a,nan,15,1990-05-15
4,July 2-3 Meeting - 1990,July,2-3,1990,n,a,2,3,1990-07-03
...,...,...,...,...,...,...,...,...,...
292,June 12-13 Meeting - 2018,June,12-13,2018,n,a,12,13,2018-06-13
293,Jul/Aug 31-1 Meeting - 2018,None,31-1,2018,Jul,Aug,31,1,2018-08-01
294,September 25-26 Meeting - 2018,September,25-26,2018,n,a,25,26,2018-09-26
295,November 7-8 Meeting - 2018,November,7-8,2018,n,a,7,8,2018-11-08


In [565]:
#df.to_csv('historic_fomc_meeting_dates.csv', index = False)

In [541]:
# Pages to scrape
url = f"https://www.federalreserve.gov/monetarypolicy/fomchistorical{2010}.htm"
url = f"https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
item = 'h5'

get_html_item(url, item)

[<h5>January 26-27 Meeting - 2010</h5>,
 <h5>March 16 Meeting - 2010</h5>,
 <h5>April 27-28 Meeting - 2010</h5>,
 <h5>May 9 Conference Call - 2010</h5>,
 <h5>June 22-23 Meeting - 2010</h5>,
 <h5>August 10 Meeting - 2010</h5>,
 <h5>September 21 Meeting - 2010</h5>,
 <h5>October 15 Conference Call - 2010</h5>,
 <h5>November 2-3 Meeting - 2010</h5>,
 <h5>December 14 Meeting - 2010</h5>]

#### 4. Web Scrape Recent Historic FOMC Meeting Dates 2019 - Until Today

In [33]:

dates = []

for year in range(2019, 2024 + 1, 1):
    # URL of the FOMC meeting calendar page
    url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"

    # Make a GET request to fetch the raw HTML content
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the header with the text "2024 FOMC Meetings"
        header = soup.find('h4', string=f"{year} FOMC Meetings")

        parent_div1 = header.find_parent('div')

        main_parent= parent_div1.find_parent('div')

        months = main_parent.find_all('div', re.compile("fomc-meeting__month "))
        days = main_parent.find_all('div', re.compile("fomc-meeting__date "))
        #days = main_parent.find_all('div', "fomc-meeting__date col-xs-4 col-sm-9 col-md-10 col-lg-1")

        #dates[year] = (months, days)
        #month_days = []
        for i in range(0, len(months)):
            
            dates.append((year,months[i].text, days[i].text))
            
        #dates[year] = month_days
        

dates[0:5]

[(2019, 'January', '29-30'),
 (2019, 'March', '19-20*'),
 (2019, 'April/May', '30-1'),
 (2019, 'June', '18-19*'),
 (2019, 'July', '30-31')]

In [34]:
df = pd.DataFrame(dates, columns=['year', 'month', 'days'])

df.head()

,year,month,days
0,2019,January,29-30
1,2019,March,19-20*
2,2019,April/May,30-1
3,2019,June,18-19*
4,2019,July,30-31


In [35]:
df['month_day1'] = df['month'].str.split('/').apply(lambda x: complete_month(x[0]) if len(x) > 1 else 'nan')
df['month_day2'] = df['month'].str.split('/').apply(lambda x: complete_month(x[1]) if len(x) > 1 else 'nan')

df['day1'] = df['days'].str.split('-').apply(lambda x: x[0] if len(x) > 1 else 'nan')
df['day2'] = df['days'].str.split('-') .apply(lambda x: x[1] if len(x) > 1 else x[0]).str.replace('(unscheduled)', '').str.replace('*', '')
df['day2'] = df['day2'].str.replace('(notation vote)', '')
df = df[~df['day2'].str.contains('cancelled')]

df.head(10)

,year,month,days,month_day1,month_day2,day1,day2
0,2019,January,29-30,nan,nan,29,30
1,2019,March,19-20*,nan,nan,19,20
2,2019,April/May,30-1,April,May,30,1
3,2019,June,18-19*,nan,nan,18,19
4,2019,July,30-31,nan,nan,30,31
5,2019,September,17-18*,nan,nan,17,18
6,2019,October,4 (unscheduled),nan,nan,nan,4
7,2019,October,29-30,nan,nan,29,30
8,2019,December,10-11*,nan,nan,10,11
9,2020,January,28-29,nan,nan,28,29


In [44]:

timestamps = []

for i, row in df.iterrows():
    
    if '/' in row['month']:
        #print('year:' , row['year'])
        #print('month:' , row['month_day2'])
        #print('day:' , int(row['day2']))
        
        timestamp = datetime.date(int(row['year']), month_str_to_num(row['month_day2']), int(row['day2']))
        timestamps.append(timestamp)
        
    else: 
        timestamp = datetime.date(int(row['year']), month_str_to_num(row['month']), int(row['day2']))
        timestamps.append(timestamp)

        
df['timestamp'] = timestamps

In [54]:
df = df[['year', 'month', 'days', 'timestamp']]
df.head()

,year,month,days,timestamp
0,2019,January,29-30,2019-01-30
1,2019,March,19-20*,2019-03-20
2,2019,April/May,30-1,2019-05-01
3,2019,June,18-19*,2019-06-19
4,2019,July,30-31,2019-07-31


In [57]:
df1 = pd.read_csv('historic_fomc_meeting_dates.csv')

df1 = df1[['year', 'month', 'day', 'timestamp']]

df1 = df1.rename(columns= {'day' : 'days'})

df1.head()

,year,month,days,timestamp
0,1990,February,6-7,1990-02-07
1,1990,March,27,1990-03-27
2,1990,April,11,1990-04-11
3,1990,May,15,1990-05-15
4,1990,July,2-3,1990-07-03


In [61]:
fomc_dates = pd.concat([df1, df], axis = 0)
fomc_dates['fomc_binary'] = 1
fomc_dates

,year,month,days,timestamp,fomc_binary
0,1990,February,6-7,1990-02-07,1
1,1990,March,27,1990-03-27,1
2,1990,April,11,1990-04-11,1
3,1990,May,15,1990-05-15,1
4,1990,July,2-3,1990-07-03,1
...,...,...,...,...,...
50,2024,June,11-12*,2024-06-12,1
51,2024,July,30-31,2024-07-31,1
52,2024,September,17-18*,2024-09-18,1
53,2024,November,6-7,2024-11-07,1


In [62]:
fomc_dates.to_csv('historic_fomc_meeting_dates_1990_2024.csv', index = False)